In [1]:
from dotenv import load_dotenv
from openpyxl.compat.singleton import Cached

load_dotenv()

True

# LocalFileStore 에서 임베딩 사용 (영구 보관)

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

# OpenAI 임베딩을 사용하여 깁노 임베딩 설정
embeddings = OpenAIEmbeddings()

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model, # 기본 임베딩과 저장소를 사용하여 캐시 지우너 임베딩을 생성
)

In [3]:
# store에서 키들을 순차적으로 가져옵니다.
list(store.yield_keys())

[]

In [10]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# 문서 로드
raw_documents = TextLoader("data/appendix-keywords.txt", encoding='utf-8').load()

# 문서 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 문서 분할
documents = text_splitter.split_documents(raw_documents)

In [11]:
from langchain.vectorstores import FAISS
# 코드 실행 시간을 측정
%time db = FAISS.from_documents(documents, cached_embedder) #문서로부터 FAISS 데이터베이스 생성

CPU times: total: 109 ms
Wall time: 1.5 s


In [12]:
# 캐시된 임베딩을 사용하여 FAISS 데이터베이스 생성
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: total: 0 ns
Wall time: 65.4 ms


# InmemoryByteStore 사용 (비영구적)

In [13]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore() # 메모리 내 바이트 저장소 생성

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings, store, namespace=embeddings.model,
)